# AcademiaOne Uni Recommender

Description...

In [2]:
import pandas as pd
import numpy as np
from scipy import sparse

from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

import pickle

## Raw data pre-processing
- load the raw data
- rename collumns
- remove the ID column
- drop incomplete entries
- split the dataset

In [3]:
df = pd.read_csv('raw_data/AO_df_15Feb_withunimajor.csv')
df.head()

,Unnamed: 0,Uni_name,Uni_Major,Uni_Jobs,Uni_Course_lectureres,Uni_Facilities,Uni_Local_life,Uni_Societies_sports
0,0,University of Leeds,Accounting and Finance BSc (Hons),2.0,1.0,0.0,4.0,4.0
1,1,University of Leeds,Geography BA (Hons),3.0,2.0,1.0,3.0,0.0
2,2,University of Leeds,Chemical Engineering MEng (Hons),4.0,4.0,0.0,0.0,1.0
3,3,University of Leeds,Genetics BSc (Hons),4.0,3.0,4.0,4.0,0.0
4,4,University of Leeds,Art and Design BA (Hons),2.0,1.0,3.0,1.0,3.0


In [4]:
df = df.rename(
    columns={
        'Uni_name':              'uni_name', 
        'Uni_Major':             'uni_major',
        'Uni_Jobs':              'job_prospects', 
        'Uni_Course_lectureres': 'course_lectures',
        'Uni_Facilities':        'facilities',
        'Uni_Local_life':        'local_life', 
        'Uni_Societies_sports':  'societies_and_sports'
        }
    )
df.head()

,Unnamed: 0,uni_name,uni_major,job_prospects,course_lectures,facilities,local_life,societies_and_sports
0,0,University of Leeds,Accounting and Finance BSc (Hons),2.0,1.0,0.0,4.0,4.0
1,1,University of Leeds,Geography BA (Hons),3.0,2.0,1.0,3.0,0.0
2,2,University of Leeds,Chemical Engineering MEng (Hons),4.0,4.0,0.0,0.0,1.0
3,3,University of Leeds,Genetics BSc (Hons),4.0,3.0,4.0,4.0,0.0
4,4,University of Leeds,Art and Design BA (Hons),2.0,1.0,3.0,1.0,3.0


In [5]:
# remove the ID columnn
df.drop(columns='Unnamed: 0', inplace=True)
df.head()

,uni_name,uni_major,job_prospects,course_lectures,facilities,local_life,societies_and_sports
0,University of Leeds,Accounting and Finance BSc (Hons),2.0,1.0,0.0,4.0,4.0
1,University of Leeds,Geography BA (Hons),3.0,2.0,1.0,3.0,0.0
2,University of Leeds,Chemical Engineering MEng (Hons),4.0,4.0,0.0,0.0,1.0
3,University of Leeds,Genetics BSc (Hons),4.0,3.0,4.0,4.0,0.0
4,University of Leeds,Art and Design BA (Hons),2.0,1.0,3.0,1.0,3.0


In [6]:
# here we drop incomplete rows
clean_data = df.copy().dropna()
clean_data.head()

,uni_name,uni_major,job_prospects,course_lectures,facilities,local_life,societies_and_sports
0,University of Leeds,Accounting and Finance BSc (Hons),2.0,1.0,0.0,4.0,4.0
1,University of Leeds,Geography BA (Hons),3.0,2.0,1.0,3.0,0.0
2,University of Leeds,Chemical Engineering MEng (Hons),4.0,4.0,0.0,0.0,1.0
3,University of Leeds,Genetics BSc (Hons),4.0,3.0,4.0,4.0,0.0
4,University of Leeds,Art and Design BA (Hons),2.0,1.0,3.0,1.0,3.0


In [7]:
# let's also do some investigation 
print('before: ' + str(df.shape[0]))
print('afer:   ' + str(clean_data.shape[0]))

# let's see the situation per column
print()
for column in df:
    print(column + ": " + str(df[column].isna().sum()))

before: 3238
afer:   2397

uni_name: 0
uni_major: 841
job_prospects: 0
course_lectures: 0
facilities: 0
local_life: 0
societies_and_sports: 0


**Observation:** As per this uni major renders a solid chunk of the dataset useless.

**Observation:** I noticed plenty of zeros in numerical features -- is that OK or should those also be taken care of?

In [8]:
# how about numerical part - what are we working with?
print(clean_data.nunique())
clean_data.describe()

uni_name                 19
uni_major               852
job_prospects             5
course_lectures           5
facilities                5
local_life                5
societies_and_sports      5
dtype: int64


,job_prospects,course_lectures,facilities,local_life,societies_and_sports
count,2397.000000,2397.000000,2397.000000,2397.000000,2397.000000
mean,2.264497,2.695035,2.554443,2.090530,1.981227
std,1.773755,1.640736,1.678966,1.839031,1.820647
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,0.000000,0.000000,0.000000
50%,3.000000,3.000000,3.000000,3.000000,3.000000
75%,4.000000,4.000000,4.000000,4.000000,4.000000
max,4.000000,4.000000,4.000000,4.000000,4.000000


In [9]:
# let's see the distribution of entries across unis
#print(df.groupby(by='uni_name')['uni_name'].count())
df['uni_name'].value_counts()

University of York                             181
Lancaster University                           181
University of Manchester                       181
University of Leeds                            181
University of Edinburgh                        181
University of Southampton                      181
University of Glasgow                          181
University of Warwick                          181
University of Sheffield                        180
University of Bristol                          180
King's College London, University of London    180
UCL (University College London)                180
Durham University                              170
University of Nottingham                       170
University of Aberdeen                         160
Queen Mary University of London                150
University of Cambridge                        150
Imperial College London                        140
University of Oxford                           130
Name: uni_name, dtype: int64

In [10]:
# ...and simiilar for majors
df.groupby(by='uni_major')['uni_major'].describe()

,count,unique,top,freq
uni_major,,,,
Accountancy and Finance MA (Hons),1,1,Accountancy and Finance MA (Hons),1
Accountancy with International Accounting BAcc (Hons),1,1,Accountancy with International Accounting BAcc...,1
Accounting BSc (Hons),3,1,Accounting BSc (Hons),3
Accounting and Finance (Industry) BSc (Hons),1,1,Accounting and Finance (Industry) BSc (Hons),1
Accounting and Finance BA (Hons),1,1,Accounting and Finance BA (Hons),1
...,...,...,...,...
War Studies BA (Hons),3,1,War Studies BA (Hons),3
War Studies MA,2,1,War Studies MA,2
World History BA (Hons),3,1,World History BA (Hons),3


**Observation:** are we missing representation for one uni or there is supposed to 19 of them?

**Observation:** (we discussed this in our call) 852 seems to be one too many distinct majors. In the description of majors we see many of the same but just different in the official title you receive -- maybe the uni_major feature could be expanded into two separate features -- e.g. "Accounting and Finance MSc" -> ("Accounting and Finance", "MSc")?

In [11]:
#X = clean_data
X = clean_data[['uni_major', 'job_prospects', 'course_lectures', 'facilities', 'local_life', 'societies_and_sports']]
X.head()

,uni_major,job_prospects,course_lectures,facilities,local_life,societies_and_sports
0,Accounting and Finance BSc (Hons),2.0,1.0,0.0,4.0,4.0
1,Geography BA (Hons),3.0,2.0,1.0,3.0,0.0
2,Chemical Engineering MEng (Hons),4.0,4.0,0.0,0.0,1.0
3,Genetics BSc (Hons),4.0,3.0,4.0,4.0,0.0
4,Art and Design BA (Hons),2.0,1.0,3.0,1.0,3.0


In [12]:
#y = clean_data.pop('uni_name')
y = clean_data['uni_name']
y.head()

0    University of Leeds
1    University of Leeds
2    University of Leeds
3    University of Leeds
4    University of Leeds
Name: uni_name, dtype: object

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=2018)

## Training the model

Current model description...

TODO:
    - ...

In [14]:
# Setting the vectorizer just like we would set a model
TfidfVec = TfidfVectorizer(
    # sublinear_tf=True,
    max_df=0.3,
    max_features=10000,
    norm='l2'
)

# Fitting the vectorizer on our training data
temp = TfidfVec.fit_transform(X['uni_major'])

fnames = TfidfVec.get_feature_names()
print(fnames)
print('num of features: ' + str(len(fnames)))
print(temp.shape)

['11', '18', 'abroad', 'accountancy', 'accounting', 'actuarial', 'administration', 'adult', 'advance', 'advanced', 'advertising', 'aerodynamics', 'aeronautical', 'aeronautics', 'aerospace', 'affairs', 'american', 'an', 'analysis', 'analytics', 'ancient', 'and', 'anglo', 'animation', 'anthropology', 'applied', 'arabic', 'archaeology', 'architectural', 'architecture', 'art', 'artificial', 'arts', 'asian', 'astronautics', 'astronomy', 'astrophysics', 'at', 'automotive', 'ba', 'bacc', 'baecom', 'banking', 'barch', 'basc', 'based', 'bass', 'bds', 'behavioural', 'beng', 'bfa', 'big', 'biochemical', 'biochemistry', 'bioengineering', 'biological', 'biology', 'biomedical', 'biomedicine', 'bioprocessing', 'biosciences', 'biotechnology', 'bmbch', 'bmbs', 'bmid', 'bmus', 'bnurs', 'brain', 'bristol', 'bs', 'bsocsc', 'business', 'bvms', 'by', 'canadian', 'cardiac', 'cdt', 'cell', 'celtic', 'certificate', 'chain', 'change', 'chb', 'chemical', 'chemistry', 'child', 'children', 'chinese', 'city', 'civi

In [15]:
preprocessor = ColumnTransformer([
    ('corpus', TfidfVec, 'uni_major'), 
    ('numerical', StandardScaler(), 
         ['job_prospects', 'course_lectures', 'facilities', 'local_life', 'societies_and_sports'])
    ], 
    remainder = 'drop', # treatment for remaining columns
    sparse_threshold=1) # set threshold for fraction of nonzero entries 
                        # to return dense array rather than sparse matrix

In [16]:
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression(penalty='none', multi_class='ovr', solver='lbfgs', max_iter=10000))])

In [17]:
# let's try to ignore the uni_major entirely
# (including the dropped entries)
temp_data = df.copy()
X1 = temp_data[['job_prospects', 'course_lectures', 'facilities', 'local_life', 'societies_and_sports']]
y1 = temp_data['uni_name']

X1_train, X1_test, y1_train, y1_test = train_test_split(
    X1, y1, test_size=0.2, stratify=y1, random_state=2018)
X1.head()
y1.head()
model = LogisticRegression(solver='lbfgs', max_iter=1000).fit(X1_train, y1_train)
model.score(X1_test, y1_test)

0.08333333333333333

In [18]:
#clf.fit(X_train, y_train)
clf.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(sparse_threshold=1,
                                   transformers=[('corpus',
                                                  TfidfVectorizer(max_df=0.3,
                                                                  max_features=10000),
                                                  'uni_major'),
                                                 ('numerical', StandardScaler(),
                                                  ['job_prospects',
                                                   'course_lectures',
                                                   'facilities', 'local_life',
                                                   'societies_and_sports'])])),
                ('classifier',
                 LogisticRegression(max_iter=10000, multi_class='ovr',
                                    penalty='none'))])

In [19]:
#cross_val_score(clf, X_train, y_train, cv=5).mean()
cross_val_score(clf, X_train, y_train, cv=5)

array([0.39583333, 0.3828125 , 0.41775457, 0.4151436 , 0.39425587])

In [20]:
clf.score(X_test, y_test)

0.45208333333333334

In [21]:
#y_train_pred = clf.predict(X_train)
#y_pred = clf.predict(X_test)

In [22]:
#y_train_pred

In [23]:
#X_train

In [24]:
#X_train.iloc[2, :]
#y
#X_test
#X_test.iloc[2, :]

In [25]:
#dataframe = pd.DataFrame(X_test.iloc[2, :])   

In [26]:
#dataframe_reshaped = dataframe.transpose(copy=False)

In [27]:
#clf.predict(dataframe_reshaped)
#X_test

## Package the model
...

In [44]:
pickle.dump(clf, open('models/uni_wizz_v0.pkl', 'wb'))

In [45]:
tmm = pickle.load(open('models/uni_wizz_v0.pkl', 'rb')) 

In [52]:
#print(type(tmm.predict(X_test.iloc[1,:])))
raw_features = {"uni_major" : "computer science", 
                "job_prospects" : 4,
                "course_lectures" : 3,
                "facilities" : 2,
                "local_life" : 3,
                "societies_and_sports" : 4}
final_features = pd.DataFrame([raw_features.values()], columns=raw_features.keys())
print(final_features)

print(str(tmm.predict(final_features)[0]))

          uni_major  job_prospects  course_lectures  facilities  local_life  \
0  computer science              4                3           2           3   

   societies_and_sports  
0                     4  
University of Warwick
